## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.hybrid.most_popular_user_CF import MostPopularCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.3)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9786567    ┆ 179524  ┆ 65.0           ┆ 100.0      │
│ 9782423    ┆ 2427481 ┆ 206.0          ┆ 100.0      │
│ 9779779    ┆ 558330  ┆ 191.0          ┆ 100.0      │
│ 9789475    ┆ 705368  ┆ 16.0           ┆ 100.0      │
│ 9772923    ┆ 87106   ┆ 45.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [3]:
recommender = MostPopularCollaborativeRecommender(train_df)
recommender.fit(scroll_weight=0.1, readtime_weight=1.0)

{179524: [(1574880, np.float64(0.999999639900517)),
  (250363, np.float64(0.999999639900517)),
  (1919068, np.float64(0.999999639900517)),
  (2576260, np.float64(0.9999996399005169)),
  (1195992, np.float64(0.9999996399005169)),
  (1664433, np.float64(0.9999996399005169)),
  (2142671, np.float64(0.9999996399005169)),
  (1482170, np.float64(0.9999996399005169)),
  (2304679, np.float64(0.9999996399005169)),
  (1281639, np.float64(0.9999996399005169))],
 2427481: [(904563, np.float64(0.5762133366403478)),
  (1904861, np.float64(0.5747344233061066)),
  (1512057, np.float64(0.5747344233061066)),
  (2558190, np.float64(0.5747344233061066)),
  (726035, np.float64(0.5747344233061066)),
  (739117, np.float64(0.5747344233061066)),
  (2138359, np.float64(0.5747343516086633)),
  (1485429, np.float64(0.574734058950197)),
  (1667778, np.float64(0.5747327258371123)),
  (2070470, np.float64(0.5706079022193139))],
 558330: [(1308040, np.float64(0.7563063737592153)),
  (2588182, np.float64(0.75630632898

This first model just compares all artilces read by users when comparing users

In [4]:
binary_recommender = MostPopularCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{179524: [(118927, np.float64(0.2940858488375231)),
  (838158, np.float64(0.2847473987257497)),
  (1612814, np.float64(0.23249527748763854)),
  (364261, np.float64(0.23249527748763854)),
  (2518915, np.float64(0.199363055707225)),
  (442840, np.float64(0.18983159915049974)),
  (1353597, np.float64(0.18983159915049974)),
  (30023, np.float64(0.18983159915049974)),
  (2040675, np.float64(0.18983159915049974)),
  (1376095, np.float64(0.18983159915049974))],
 2427481: [(911657, np.float64(0.5163977794943222)),
  (822785, np.float64(0.44721359549995787)),
  (1282527, np.float64(0.44721359549995787)),
  (250154, np.float64(0.44721359549995787)),
  (1208893, np.float64(0.44721359549995787)),
  (1771774, np.float64(0.44721359549995787)),
  (491535, np.float64(0.44721359549995787)),
  (1617453, np.float64(0.44721359549995787)),
  (1904861, np.float64(0.44721359549995787)),
  (1512057, np.float64(0.44721359549995787))],
 558330: [(1397660, np.float64(0.2581988897471611)),
  (1594362, np.float64(

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [5]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9778351, 9771168, 9781878, 9772813, 9778386]
reccomended for user  620796 :  [9771224, 9772706, 9778219, 9766752, 9780406]
reccomended for user  1067393 :  [9771113, 9789810, 9771627, 9769424, 9786159]
reccomended for user  1726258 :  [9773282, 9776234, 9775776, 9782423, 9785475]
reccomended for user  17205 :  [9780325, 9774764, 9781991, 9778939, 9770886]


In [6]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9775484, 9783334, 9769504, 9775518, 9771948]
reccomended for user  620796 :  [9780280, 9783278, 9784559, 9783865, 9771224]
reccomended for user  1067393 :  [9771113, 9787098, 9770082, 9778710, 9744403]
reccomended for user  1726258 :  [9773282, 9776234, 9775776, 9782423, 9785475]
reccomended for user  17205 :  [9775562, 9780325, 9783657, 9783024, 9771846]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [7]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=False)
results

{'MAP@K': np.float64(0.005454545454545455),
 'NDCG@K': np.float64(0.06001039270384032)}

The binary reccomender model only reccomending articles the user has not yet read

In [8]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=False)
results

{'MAP@K': np.float64(0.006923076923076922),
 'NDCG@K': np.float64(0.03474426203319197)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [9]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=True)
results

{'MAP@K': np.float64(0.007692307692307693),
 'NDCG@K': np.float64(0.047746347613807034)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [10]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=20, allow_read_articles=True)
results

{'MAP@K': np.float64(0.004545454545454546),
 'NDCG@K': np.float64(0.05000327672518417)}

## Model Experimentation

In [11]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9772045, 9773711, 9784591, 9774352, 9776916, 9776406, 9789473, 9774764, 9769531, 9672256, 9772355, 9538375, 9789896, 9788362, 9782092, 9771473, 9780181, 9772508, 9776862, 9778657, 9783655, 9786351, 9776497, 9781875, 9781878, 9787510, 9673979}
[9778351, 9771168, 9781878, 9772813, 9778386, 9779867, 9782884, 9776560, 9772095, 9781998, 9774972, 9786378, 9778035, 9772475, 9775983, 9790784, 9777621, 9769917, 9776041, 9780384, 9789647, 9774383, 9783276, 9772963, 9775562, 9785668, 9781913, 9780960, 9773574, 9776497, 9772088, 9783732, 9771919, 9770327, 9784444, 9781598, 9787332, 9771916, 9773282, 9776234, 9775776, 9782423, 9785475, 9787465, 9790548, 9781086, 9778168, 9789810, 9779737, 9779748, 9782517, 9787524, 9780195, 9780280, 9782092, 9781476, 9789065, 9783213, 9786932, 9770082, 9784856, 9776508, 9773846, 9774079, 9783057, 9783278, 9775484, 9779289, 9774074, 9778804, 9785835, 9784782, 9784591, 9774142, 9773744, 9783752, 9782407, 9773278, 9776322, 9779538, 9785310, 9784947,

In [12]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9772045, 9773711, 9784591, 9774352, 9776916, 9776406, 9789473, 9774764, 9769531, 9672256, 9772355, 9538375, 9789896, 9788362, 9782092, 9771473, 9780181, 9772508, 9776862, 9778657, 9783655, 9786351, 9776497, 9781875, 9781878, 9787510, 9673979}
[9778351, 9771168, 9781878, 9772813, 9778386, 9779867, 9782884, 9776560, 9772095, 9781998, 9774972, 9786378, 9778035, 9772475, 9775983, 9790784, 9777621, 9769917, 9776041, 9780384, 9789647, 9774383, 9783276, 9772963, 9775562, 9781913, 9785668, 9780960, 9773574, 9776497, 9772088, 9783732, 9771919, 9770327, 9784444, 9781598, 9787332, 9771916, 9773282, 9776234, 9775776, 9782423, 9785475, 9787465, 9790548, 9781086, 9778168, 9789810, 9779737, 9779748, 9782517, 9787524, 9780195, 9780280, 9782092, 9781476, 9789065, 9783213, 9786932, 9770082, 9784856, 9776508, 9773846, 9774079, 9783057, 9783278, 9775484, 9779289, 9774074, 9778804, 9785835, 9784782, 9784591, 9774142, 9773744, 9783752, 9782407, 9773278, 9776322, 9779538, 9785310, 9784947,

In [13]:
from utils.evaluation import perform_model_evaluation

matrics = perform_model_evaluation(recommender, test_df, k=5)
matrics

{'precision@k': np.float64(0.00651497005988024),
 'recall@k': np.float64(0.010266810272819565),
 'fpr@k': np.float64(0.0022638758971479916)}

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [14]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "user_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 12:08:20] [setup] RAM Tracking...
[codecarbon INFO @ 12:08:20] [setup] CPU Tracking...
[codecarbon WARNING @ 12:08:21] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon INFO @ 12:08:22] CPU Model on constant consumption mode: AMD Ryzen 5 5500U with Radeon Graphics
[codecarbon INFO @ 12:08:22] [setup] GPU Tracking...
[codecarbon INFO @ 12:08:24] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:08:24] >>> Tracker's metadata:
[codecarbon INFO @ 12:08:24]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 12:08:24]   Python version: 3.11.1
[codecarbon INFO @ 12:08:24]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 12:08:24]   Available RAM : 7.338 GB
[codecarbon INFO @ 12:08:24]   CPU count: 12
[codecarbon INFO @ 12:08:24]   CPU model: AMD Ryzen 5 5500U with Radeon Graphics
[codecarbon INFO @ 12:08:24]   GPU count: 1
[codecarbon INFO @ 12:08:24]   GPU model: 1 x NVIDIA GeForce GTX 1650
[codecarbon INFO @ 12:08:24] Saving emissions data to file c:\Users\chris\Desktop\NTNU Ting\8. Semester\Anbefalingssystemer\Project\TDT4215\recommender_system\demostrations\output\user_based_fit_emission.csv
[codecarbon INFO @ 12:08:39] Energy c

{'fit': ({179524: [(1195992, np.float64(0.9999996398801758)),
    (1482170, np.float64(0.9999996398801758)),
    (1574880, np.float64(0.9999996398801758)),
    (1919068, np.float64(0.9999996398801757)),
    (2576260, np.float64(0.9999996398801757)),
    (1664433, np.float64(0.9999996398801757)),
    (2142671, np.float64(0.9999996398801757)),
    (1824596, np.float64(0.9999996398801757)),
    (2304679, np.float64(0.9999996398801757)),
    (1281639, np.float64(0.9999996398801757))],
   2427481: [(911657, np.float64(0.5385291264999326)),
    (904563, np.float64(0.535645482006204)),
    (1512057, np.float64(0.5324800008676542)),
    (1904861, np.float64(0.5324800008676542)),
    (726035, np.float64(0.5324800008676542)),
    (739117, np.float64(0.5324800008676542)),
    (2558190, np.float64(0.5324800008676542)),
    (2138359, np.float64(0.5324799189895141)),
    (1667778, np.float64(0.532478066380492)),
    (1485429, np.float64(0.5324719241494574))],
   558330: [(1308040, np.float64(0.74345